In [1]:
import pandas
import time
import github
import gpt
import Score

In [4]:
def run_experiment(csvFile):
    for i in range(0,5):
        repo_identifier = csvFile.iloc[i]['GitHub_Repo_Link'].split('github.com/')[1]
        print(repo_identifier)
        try:
            default_branch = github.get_default_branch(repo_identifier);
            repo_structure = github.get_repository_tree(repo_identifier, default_branch)
            dependencies = github.get_list_of_dependencies(repo_identifier)

            generated_workflow_file = gpt.generate_build_pipeline(repo_structure, dependencies, default_branch)

            valid_syntax = github.is_workflow_syntax_valid(generated_workflow_file);
            if not valid_syntax:
                csvFile.loc[i,'Syntax_Check'] = 'Invalid'
                continue

            csvFile.loc[i,'Syntax_Check'] = 'Valid'
            csvFile.loc[i,'Generated_Build_Pipeline_File_Content'] = generated_workflow_file
            workflow_files = github.get_all_workflow_files(repo_identifier)

            build_file_content = ''
            build_file_match_score = 0
            build_file_bleu_score = 0
            # Loop through workflow files
            for workflow_file in workflow_files:
                workflow_file_content = github.get_workflow_file_content(repo_identifier, workflow_file, default_branch)
                exact_match_score = Score.get_exact_match_score(generated_workflow_file, workflow_file_content)
                bleu_score = Score.get_bleu_score(generated_workflow_file, workflow_file_content)
                if exact_match_score + bleu_score > build_file_match_score + build_file_bleu_score:
                    build_file_match_score = exact_match_score
                    build_file_content = workflow_file_content
                    build_file_bleu_score = bleu_score

            csvFile.loc[i,'GitHub_Build_Pipeline_File_Content'] = build_file_content
            csvFile.loc[i,'BLEU_Score'] = build_file_bleu_score
            csvFile.loc[i,'Exact_Match_Score'] = build_file_match_score
            # Add delay to avoid rate limiting
            time.sleep(10)
        except Exception as e:
            print(e)
            continue

    csvFile.to_csv('dataset/output.csv', index=False)

In [3]:
# reading the CSV file
csvFile = pandas.read_csv('dataset/dataset_sample.csv')

# Pre-processing the CSV file
csvFile = csvFile[csvFile['GitHub_Repo_Link'].notna()]

csvFile

,Index,GitHub_Repo_Link,GitHub_Build_Pipeline_File_Content,Generated_Build_Pipeline_File_Content,Exact_Match_Score,BLEU_Score,Syntax_Check,DevOps_Aware_Score
0,1.0,https://github.com/DapperLib/Dapper,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,https://github.com/huiyadanli/RevokeMsgPatcher,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,https://github.com/jasontaylordev/CleanArchite...,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,https://github.com/nilaoda/N_m3u8DL-CLI,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,https://github.com/shadowsocks/shadowsocks-win...,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
run_experiment(csvFile)

DapperLib/Dapper


/tmp/ipykernel_38826/3603651508.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Valid' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  csvFile.loc[i,'Syntax_Check'] = 'Valid'
/tmp/ipykernel_38826/3603651508.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'name: Build and Test

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  build:
    runs-on: ubuntu-latest

    steps:
    - name: Checkout repository
      uses: actions/checkout@v2

    - name: Setup .NET Core
      uses: actions/setup-dotnet@v1
      with:
        dotnet-version: 3.1

    - name: Restore dependencies
      run: dotnet restore

    - name: Build
      run: dotnet build --configuration Release

    - name: Test
      run: |
        if [ -f "./tests" ]; then
          dotnet test
        else
 

huiyadanli/RevokeMsgPatcher
jasontaylordev/CleanArchitecture
nilaoda/N_m3u8DL-CLI
shadowsocks/shadowsocks-windows
